In [ ]:
!pip install scapy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 29.8 MB/s eta 0:00:00


In [ ]:
### Breeze-v0.0.1 Test

#===Imports
from scapy.all import rdpcap, IP, TCP, UDP, DNS, DNSQR
import pandas as pd
from collections import Counter
#===

#===Globals
path = './ping_dns_test.pcapng'
#===

## Testing PCAP access
print(f'[-] Accessing PCAP at path: {"{"}{path}{"}"}`... ', end='')
try:
  packets = rdpcap(path)
  print(f'DONE.')
except Exception as e:
  print(f'\n\t[!] Could not access PCAP at path: `{path}`. Aborting...')
  exit(0)

### Getting information about PCAP
# rows, start/end time, date

[-] Accessing PCAP at path: {./ping_dns_test.pcapng}`... DONE.


In [ ]:
for pkt in packets:
  print(pkt)

Ether / IP / TCP 20.69.136.49:443 > 66.71.101.209:54089 FA
Ether / IP / TCP 66.71.101.209:54089 > 20.69.136.49:443 A
Ether / IP / TCP 162.159.130.234:443 > 66.71.101.209:53941 PA / Raw
Ether / IP / TCP 162.159.130.234:443 > 66.71.101.209:53941 PA / Raw
Ether / IP / TCP 66.71.101.209:53941 > 162.159.130.234:443 A
Ether / IP / TCP 162.159.130.234:443 > 66.71.101.209:53941 PA / Raw
Ether / IP / TCP 66.71.101.209:53941 > 162.159.130.234:443 A
Ether / IP / TCP 162.159.130.234:443 > 66.71.101.209:53941 PA / Raw
Ether / IP / TCP 66.71.101.209:53941 > 162.159.130.234:443 A
Ether / IP / TCP 162.159.130.234:443 > 66.71.101.209:53941 PA / Raw
Ether / IP / TCP 66.71.101.209:53941 > 162.159.130.234:443 A
Ether / IP / TCP 162.159.130.234:443 > 66.71.101.209:53941 PA / Raw
Ether / IP / TCP 66.71.101.209:53941 > 162.159.130.234:443 A
Ether / IP / TCP 162.159.130.234:443 > 66.71.101.209:53941 PA / Raw
Ether / IP / TCP 66.71.101.209:53941 > 162.159.130.234:443 A
Ether / IP / TCP 66.71.101.209:53958 > 17

In [ ]:
from datetime import datetime

timestamp = packets[0].time
converted_timestamp = datetime.fromtimestamp(timestamp)
print(f'original : {timestamp}')
print(f'converted: {converted_timestamp}')

TypeError: 'EDecimal' object cannot be interpreted as an integer

In [ ]:
from datetime import datetime

datetime.fromtimestamp(1770132796.3301687)

datetime.datetime(2026, 2, 3, 15, 33, 16, 330169)

# working time stamp comparison
but without seconds conversion. need to continue with UTC date timestamp.

In [ ]:
from datetime import datetime

p0 = packets[0]
pLAST = packets[len(packets) - 1]
p0_t = p0.time
pLAST_t = pLAST.time

delta = pLAST_t - p0_t
print(f'delta p0, pLAST: {float(delta):.6f} seconds')

delta p0, pLAST: 77.720585 seconds


In [ ]:
# sample

from scapy.all import rdpcap, IP, TCP, UDP, DNS, DNSQR
import pandas as pd
from collections import Counter

PCAP_FILE = "cap.pcapng"

# ----------------------------
# Load PCAP
# ----------------------------
packets = rdpcap(PCAP_FILE)

rows = []
iocs = {
    "ips": set(),
    "domains": set(),
    "ports": set()
}

# ----------------------------
# Packet parsing
# ----------------------------
for pkt in packets:
    if IP in pkt:
        src_ip = pkt[IP].src
        dst_ip = pkt[IP].dst
        proto = pkt[IP].proto

        src_port = None
        dst_port = None
        dns_query = None

        # TCP
        if TCP in pkt:
            src_port = pkt[TCP].sport
            dst_port = pkt[TCP].dport
            iocs["ports"].add(dst_port)

        # UDP
        elif UDP in pkt:
            src_port = pkt[UDP].sport
            dst_port = pkt[UDP].dport
            iocs["ports"].add(dst_port)

        # DNS
        if pkt.haslayer(DNS) and pkt.haslayer(DNSQR):
            dns_query = pkt[DNSQR].qname.decode(errors="ignore")
            iocs["domains"].add(dns_query)

        # Track IP IOCs
        iocs["ips"].update([src_ip, dst_ip])

        rows.append({
            "src_ip": src_ip,
            "dst_ip": dst_ip,
            "src_port": src_port,
            "dst_port": dst_port,
            "protocol": proto,
            "dns_query": dns_query,
            "packet_len": len(pkt)
        })

for pkt in packets:
  print(pkt)

# ----------------------------
# Build DataFrame
# ----------------------------
df = pd.DataFrame(rows)

# ----------------------------
# Feature analysis
# ----------------------------
top_src_ips = Counter(df["src_ip"]).most_common(10)
top_dst_ips = Counter(df["dst_ip"]).most_common(10)
top_ports = Counter(df["dst_port"].dropna()).most_common(10)
top_domains = Counter(df["dns_query"].dropna()).most_common(10)

# ----------------------------
# Output summary
# ----------------------------
print("\n=== IOC SUMMARY ===")
print(f"Unique IPs: {len(iocs['ips'])}")
print(f"Unique Domains: {len(iocs['domains'])}")
print(f"Unique Ports: {len(iocs['ports'])}")

print("\nTop Source IPs:")
for ip, count in top_src_ips:
    print(f"{ip}: {count}")

print("\nTop Destination IPs:")
for ip, count in top_dst_ips:
    print(f"{ip}: {count}")

print("\nTop Destination Ports:")
for port, count in top_ports:
    print(f"{port}: {count}")

print("\nTop Queried Domains:")
for domain, count in top_domains:
    print(f"{domain}: {count}")

# ----------------------------
# Save for analysis / ML
# ----------------------------
df.to_csv("flow_packet_features.csv", index=False)
```